In [113]:
#read data
def readData(gap,dataset):
  # Dataset number. 1, 2, 3 or 4
  filename = "gap" + gap + "-" + dataset + ".txt"
  f = open(filename, 'r')
  l = f.readline().split()

  # number of agents
  agents = int(l[0])

  # number of jobs
  jobs = int(l[1])

  # cost of assigning job j to agent i
  costs = []

  # resource required by agent i to perform job j
  resource_of_agent_job = []

  # resource capacity of agent i.
  capacity_of_agents = []


  for i in range(agents):
      temp = []
      for l in f.readline().split():
          temp.append(int(l))
      costs.append(temp)
      
  for i in range(agents):
      temp = []
      for l in f.readline().split():
          temp.append(int(l))
      resource_of_agent_job .append(temp)

  for l in f.readline().split():
      capacity_of_agents.append(int(l))
      
  f.close()

  print("AGENTS", agents)
  print("JOBS", jobs)
  print("COSTS", costs)
  print("resource_of_agent_job", resource_of_agent_job)
  print("capacity_of_agents", capacity_of_agents)
  array = []
  array.append(agents)
  array.append(jobs)
  array.append(costs)
  array.append(resource_of_agent_job)
  array.append(capacity_of_agents)
  return array


In [114]:
import copy
import math
import random
def simulated_annealing(agents,jobs,resource_of_agent_job,capacity_of_agents):
    """Peforms simulated annealing to find a solution"""
    initial_temp = agents * jobs
    final_temp = .1
    alpha = 0.001
    iteration = 0
    current_temp = initial_temp
    flag = True
    # Start by initializing the current state with the initial state
    current_solution = init_solution(current_temp,jobs,agents)
    print("Initial SOL", current_solution, "Cost", get_cost(current_solution), end =" | ")
    
    solution = current_solution
    bestSolution = solution
    while current_temp > final_temp and iteration < 100000:
        neighbor = findNeighbor(solution,current_temp,jobs,agents)

        # Check if neighbor is best so far
                     
        cost_diff = get_cost(neighbor) - get_cost(solution) - calculateUnfit(neighbor)

        # if the new solution is better, accept it
        if cost_diff > 0:
            solution = neighbor
        # if the new solution is not better, accept it with a probability of e^(-cost/temp)
        else:
            
            if random.uniform(0, 1) < ((math.exp(cost_diff / current_temp))):
                
                solution = neighbor
        # decrement the temperature
        current_temp -= (current_temp * alpha)
        #print("FEA",isFeasible(solution),">",get_cost(solution) > get_cost(bestSolution))

        if isFeasible(solution) and  get_cost(solution) > get_cost(bestSolution):
          bestSolution = list(solution)
          
        iteration = iteration + 1
        
    #print("Iteration", iteration, "Final Sol", solution, "Cost", get_cost(solution))
    #print(isFeasible(solution))
    return bestSolution,get_cost(bestSolution)
    

def init_solution(current_temp,jobs,agents):
    solution = []
    for agent in range(0,agents):
        total_cost = 0
        mirrored = copy.deepcopy(resource_of_agent_job[agent])
        tempSol = []
        #print("CAPACITY OF AGENTS", agent , " ", capacity_of_agents[agent])
        i = 0
        while capacity_of_agents[agent] >= total_cost and i < (jobs/agents):
          
          min_value = min(mirrored)
          min_index = mirrored.index(min_value)
          mirrored[min_index] = 999999
          if min_value == 999999:
            break
          if searchJob(min_index,solution):
            tempSol.append(min_index)
            total_cost += min_value
            i = i + 1

        solution.append(tempSol)
    
    solution = turnIntoJobBase(solution,jobs)
    print("Solution Before Init Neighbor" , solution, "Cost" , get_cost(solution), end =" | ")
    solution = findNeighbor(solution,current_temp,jobs,agents)
    while isFeasible(solution) == False:
      solution = findNeighbor(solution,current_temp,jobs,agents)

    return solution


def turnIntoJobBase(solution,jobs) :
    array = [0] * jobs
    for singleSol in solution:
        for index in singleSol:
          array[index] = solution.index(singleSol)
    
    return array

def searchJob(min_index, solution):
    for singleSol in solution:
        for index in singleSol:
          if min_index == index:
            return False
    return True

def get_cost(solution):
    cost = 0
    for i in range(len(solution)):
        cost = cost + costs[solution[i]][i]
    return cost


def calculateUnfit(solution):
    unfitness = 0
    caps = [0] * agents 
    
    for i in range(len(solution)):
        caps[solution[i]] = caps[solution[i]] + resource_of_agent_job[solution[i]][i]
        
    for i in range(len(caps)):
        unfitness = unfitness + max(0, caps[i] - capacity_of_agents[i])
    
    return unfitness*16

def isFeasible(solution):
    caps = [0] * agents
    for i in range(len(solution)):
        caps[solution[i]] = caps[solution[i]] + resource_of_agent_job[solution[i]][i]
    
    for i in range(len(caps)):
        if capacity_of_agents[i] < caps[i]:
            return False
     
    return True


def findNeighbor(solution, current_temp, jobs,agents):
    nSize = math.ceil(current_temp / jobs)
    newSolution = list(solution)
    for i in range(nSize*2):
        random_job_two = random.randint(0,jobs-1) 
        newSolution[random_job_two] = random.randint(0,agents-1)
    return newSolution


In [118]:
agents = 0
jobs = 0

  # cost of assigning job j to agent i
costs = []

# resource required by agent i to perform job j
resource_of_agent_job = []

# resource capacity of agent i.
capacity_of_agents = []

sum = 0
for gapID in range(4,5):
  for instanceID in range(4,6):
    print("gap" + str(gapID) + "-" + str(instanceID))
    array = readData(str(gapID),str(instanceID))
    agents= array[0]
    jobs = array[1]
    costs = array[2]
    resource_of_agent_job = array[3]
    capacity_of_agents = array[4]
    for i in range(10) :
        sol,cost = simulated_annealing(agents,jobs,resource_of_agent_job,capacity_of_agents)
        map = dict()
        for i in range(len(sol)):
          if str(sol[i]) in map:
            map[str(sol[i])] += resource_of_agent_job[sol[i]][i]
          else : 
            map[str(sol[i])] = resource_of_agent_job[sol[i]][i]
        dictionary_items = map.items()
        sorted_items = sorted(dictionary_items)
        print( "Solution" , sol, "Cost", cost , "Resources Of Agents", sorted_items,end =" | ")
        print(";")
    
  

print(sum/10)

gap4-4
AGENTS 5
JOBS 30
COSTS [[16, 14, 16, 21, 13, 14, 14, 21, 20, 19, 25, 12, 22, 17, 12, 21, 11, 10, 24, 15, 23, 14, 14, 20, 15, 23, 14, 23, 13, 22], [24, 18, 23, 18, 12, 20, 12, 15, 14, 22, 22, 18, 18, 21, 12, 19, 18, 13, 16, 16, 16, 10, 18, 15, 14, 13, 23, 15, 12, 10], [20, 16, 15, 18, 24, 16, 21, 21, 24, 15, 14, 23, 11, 12, 15, 14, 18, 15, 14, 12, 25, 20, 23, 13, 16, 23, 18, 20, 22, 16], [20, 14, 20, 16, 17, 24, 15, 23, 13, 11, 18, 22, 10, 15, 19, 10, 21, 10, 21, 11, 13, 16, 18, 22, 20, 16, 21, 22, 19, 20], [19, 15, 12, 11, 23, 24, 15, 17, 17, 11, 23, 14, 20, 21, 22, 16, 16, 23, 10, 22, 12, 25, 24, 20, 13, 20, 16, 11, 23, 18]]
resource_of_agent_job [[15, 16, 24, 17, 9, 18, 5, 8, 25, 21, 22, 22, 11, 7, 23, 15, 16, 19, 22, 21, 6, 14, 13, 19, 21, 14, 21, 15, 17, 10], [22, 13, 7, 14, 23, 10, 6, 11, 21, 14, 15, 9, 13, 18, 15, 14, 7, 7, 17, 8, 16, 19, 16, 15, 10, 7, 12, 10, 21, 13], [9, 16, 13, 20, 10, 17, 16, 15, 24, 12, 16, 8, 6, 6, 21, 23, 6, 16, 9, 18, 20, 10, 12, 15, 20, 21, 11, 9